In [13]:
import numpy as np 
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import rc 
import os, shutil 
import sqa
from BTK_fit import *
from scipy.optimize import least_squares
%matplotlib qt

In [8]:
IV=BTK_IV(2,1,5,0.5,3) #def BTK_IV(T,delta,Z,gamma,Vrange):
#IV1=BTK_IV(2,1,1,0.00,2)
#IV2=BTK_IV(2,1,1.5,0.00,2)
#IV3=BTK_IV(2,1,2,0.00,2)



fig=plt.figure(figsize=(8,6))
# plt.axes([0.18,0.18,0.75,0.7])




plt.plot(IV[:,0],IV[:,1],'-k')
#plt.plot(IV1[:,0],IV1[:,1],'-r')
#plt.plot(IV2[:,0],IV2[:,1],'-m')
#plt.plot(IV3[:,0],IV3[:,1],'-g')


plt.xlabel('Bias Voltage (mV)')
plt.ylabel('($\mathit{dI/dV}$)$_{\mathdefault{n}}$')
plt.legend(('Z=0.5'),loc=0,frameon=False,prop={'size':12})
plt.xlim(-2,2)
plt.ylim(0,2.1)
#plt.text(r'T=2 K;Z=0',ha='left',color='r')
#plt.text(r'$\Delta$=1 meV',ha='left',color='r')
#plt.ylim(0.87,1.08)$\Delta$=1 meV

#fname = 'S2.pdf'

#plt.savefig(fname,format='pdf')

plt.show()

In [44]:
temp=2
gap=1
barrier=1
gamma=0
Vrange=5
def BTK_fitting(fitV,fitG,T=temp,GAP=gap,B=barrier,GA=gamma,V=Vrange,p0=False):
    '''
    
    P[0]: delta/mV, gap;
    P[1]: Z, barrier;
    P[0]: gamma/mV;
    
    '''
    def btk_fitG(T,GAP,B,GA,V,y):
        return btk_IV(T,GAP,B,GA,V)-y
    
    def btk_IV(T,GAP,B,GA,V):
        T=np.float(T); delta=np.float(GAP);Z=np.float(B);gamma=np.float(GA); Vrange=np.float(V)
        energy_range=3*Vrange # the range over which we want to find G(E), unit is mV.
        nenergy_vals=301 # the number of different energy values, related to the resolution of integration.
        energy_step=energy_range/nenergy_vals
        volt_range=Vrange 
        nvolts=601 # the number of voltage steps
        volt_step=volt_range/nvolts 
        # Declare arrays to hold the integration results
        Energy_vals=np.zeros(nenergy_vals)
        IntegrandofE=np.zeros(nenergy_vals)
        IntegrandofE2=np.zeros(nenergy_vals)
        # Define variable arrays 
        energy_vals_array=np.linspace(1,nenergy_vals,num=nenergy_vals)
        Energy_vals=(energy_vals_array*energy_step)-energy_range/2.
        volt_array=np.zeros(nvolts)
        IofV=np.zeros(nvolts)
        GofV=np.zeros(nvolts)
        for j1 in range(nvolts):
            volt_array[j1]=j1*volt_step
            for k in range(nenergy_vals):
                E=Energy_vals[k]    
                IntegrandofE2[k]=(1.+Z**2)*deriv_fermi(E-volt_array[j1], T)*(1.+BTK_AB(E, delta, Z,gamma)[0]-BTK_AB(E, delta, Z,gamma)[1])
                IntegrandofE[k]=(1.+Z**2)*(fermi(E-volt_array[j1], T)-fermi(E, T))*(1.+BTK_AB(E, delta, Z,gamma)[0]-BTK_AB(E, delta, Z,gamma)[1])
            GofV[j1]=np.trapz(IntegrandofE2,x=Energy_vals)
            IofV[j1]=np.trapz(IntegrandofE,x=Energy_vals)
        volt_array=np.append(volt_array[::-1]*(-1),volt_array[1:])
        GofV=np.append(GofV[::-1],GofV[1:])
        IofV=np.append(IofV[::-1]*(-1),IofV[1:])
        
        return GofV
    
    
    if not p0:
        p0=p0
    else:
        p0=[T,GAP,B,GA]
        
    res_robust=least_squares(btk_fitG,p0,loss='soft_l1',f_scale=0.1,args=(fitV,fitG))
    fitted_G=btk_IV(res_robust.x,fitV)

    return res_robust.x,fitted_G
    

In [45]:
def fit_BTK_curve(curvex,curvey,T=2,GAP=1,B=1,GA=0,width=[400,400],figure_path='C:\\Users\\QSong_pku\\Desktop\\temp.png'):
    
	#global figure_path
	
	if np.max(np.abs(curvex))<100:
		curvex=curvex*1000
	#plt.figure()
	#plt.figure()
	plt.ion()
	plt.plot(curvex,curvey,'r.')
	plt.title(figure_path)
	plt.draw()
	plt.pause(0.1)
	
	while True:
		print \
		'''
		no point => ok 
		1 point => center
		3 point => limit the left and right width
		>4 point => repeat
        5 point => give up
		'''
		try:
			ginput=plt.ginput(0)
		except:
			continue
            
		if len(ginput)==3:
			refinedx,refinedy=sqa.fit_refine_data(curvex,curvey,ginput[0][0],ginput[2][0])
			plt.clf()
			plt.plot(refinedx,refinedy,'r.',ms=10)
			plt.draw()
			plt.pause(0.1)
			x0=[T,(np.abs(ginput[2][0])+np.abs(ginput[0][0]))/2,B,GA]
			parameter,fitted_y=BTK_fitting(refinedx,refinedy,T=x0[0],GAP=x0[1],B=x0[2],GA=x0[3])
			#plt.plot(refinedx,refinedy,'.')
			plt.plot(refinedx,fitted_y,'b.',ms=15)
			plt.title(figure_path)
			plt.draw()
			plt.pause(0.1)
			#plt.pause(0.1)
		elif len(ginput)==0:
			break
		elif len(ginput)==4:
			plt.clf()
			plt.plot(curvex,curvey,'r.')
			plt.title(figure_path)
			plt.draw()
			plt.pause(0.1)
		elif len(ginput)>4:
			parameter=np.zeros(6)
			break
		else:
			print 'ginput error'
		#plt.savefig(figure_path)
	plt.clf()
	return parameter

In [46]:
folder='E:\\project\\Nontrival SC\\SF_CRR'
filename='S3_1.txt'
data=sqa.fmr_load_data(folder+'\\'+filename)


In [47]:
plt.figure()
plt.plot(data.index*1000,data.values,'r-')

In [48]:
fitting_para=fit_BTK_curve(data.index.values*1000,data.values,T=2,GAP=1,B=1,GA=0,width=[400,400],figure_path='C:\\Users\\QSong_pku\\Desktop\\temp.png')

print(fitting_para)


		no point => ok 
		1 point => center
		3 point => limit the left and right width
		>4 point => repeat
        5 point => give up
		


TypeError: btk_fitG() takes exactly 6 arguments (3 given)